## Building a nanoGPT

Companion notebook to the [Zero To Hero](https://karpathy.ai/zero-to-hero.html) video on GPT.

# https://www.youtube.com/watch?v=kCc8FmEb1nY

In [ ]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
#!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
# download need to FQ, here I just copy both txt files from git

In [ ]:
EXP_LEN = 200 
SEED = 1337

In [ ]:
import torch # we use PyTorch: https://pytorch.org
# print(torch.cuda.is_available())

import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(SEED)

In [ ]:
def check_gpu_avlailabe():
    return torch.cuda.is_available()

In [ ]:
def move_to_gpu(Model, data, targets):
    device = torch.device("cuda")
    model  = Model.to(device)
    data = data.to(device)
    targets = targets.to(device)
    return device

## Prepare the Train / Val dataset

In [ ]:
# read text data
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [ ]:
print("length of dataset in characters: ", len(text))

In [ ]:
# let's look at the first #EXP_LEN characters
print(text[:EXP_LEN])

In [ ]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(f"unique token number: {vocab_size}") # show all embedding token

In [ ]:
# create a mapping from characters to integers
txt = "hii there"

stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }

encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(f"chars[46] is {chars[46]}; chars[47] is {chars[47]}")

print(f"encode result is : {encode(txt)}") # h : 46, i : 47, 'space' = 1 etc.. actually the index of the chars (#46 is 'h')
print(f"decode result is : {decode(encode(txt))}")

In [ ]:
# using google's sentencepiece
# using openai's  tiktoken
import tiktoken as tt
enc_gpt2 = tt.get_encoding('gpt2')  # take gpt2 encode
print(enc_gpt2.n_vocab)  # largest token? 
print(enc_gpt2.encode(txt))
print(enc_gpt2.decode(enc_gpt2.encode(txt)))

In [ ]:
# let's now encode the entire text dataset and store it into a torch.Tensor

data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(text[:EXP_LEN])
print("=" *30)
print(data[:EXP_LEN]) # the EXP_LEN characters we looked at earier will to the GPT look like this

In [ ]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val,
train_data = data[:n]
val_data = data[n:]

In [ ]:
block_size = 8  # the chunk size of the block that once time feed into the transfomer.
print(f"encoded train[0:{block_size}]")
print(train_data[:block_size+1])
print(f"encoded val[0:{block_size}]")
print(val_data[:block_size+1])

print(f"decoded train[0:{block_size}]")
print(decode(train_data[:block_size+1].tolist()))
print(f"decoded val[0:{block_size}]")
print(decode(val_data[:block_size+1].tolist()))

In [ ]:
# demo how a train block works, a train block has total 9 token
x = train_data[:block_size]  # x is input 
y = train_data[1:block_size+1] # y is target 
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

In [ ]:
torch.manual_seed(SEED)
demo_batch_size = 4 # how many independent sequences will we process in parallel?
demo_block_size = 8 # what is the maximum context length for predictions?

# print(torch.randint(len(data) - block_size, (demo_batch_size, demo_batch_size+2, )))

def get_batch(split, batch_size, block_size):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data  # select train_data or val_data by input 
    ix = torch.randint(len(data) - block_size, (batch_size,))  # a random index list, 
    # len(data) - block_size make sure idex does not exceed the end of the data array minus one block's worth of elements 
    # (batch_size,) give shape of the output tensor, and this is one dimension array
    # print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train', demo_batch_size, demo_block_size)
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)
print('target looks like input + 1 shift ...')
print('----')

for b in range(demo_batch_size): # batch dimension
    #print(f"batch #{b}")
    ti = 0
    for t in range(demo_block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        # print(f"when input is {context.tolist()} the target: {target}")

In [ ]:
print(xb) # our input to the transformer

## Now let us create a LLM model and feed the train sequece to it.

In [ ]:
# A very simple model

class HandsOn(nn.Module):
    
    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size) # create embedding table with 65x65
        print(f"embedding_table dimension : {self.token_embedding_table}")
        #print(self.token_embedding_table)

    def forward(self, idx, targets=None):
        # idx: identity? train data (input)
        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C) # B: batch, T: Time C: Channel of tensor
        print(idx)        
        # print(logits)
        B, T, C = logits.shape
        print(f"Batch 行数 B = {B}, Time 列数 T = {T}, Channel 每个embedding的维度 C = {C}")
        return logits

ms = HandsOn(vocab_size) # create a model instance, vocab_size is # of embedding, 65 for this example
# call forward, idx = xb, targets = yb
ms_out = ms(xb, yb)

print(ms_out.shape) # [4, 8, 65], 即4行，每行8列，共32个embedding token，每个embedding token为65维度的vector

print(ms_out) # print all tokens
#pront the batch 0, #0 token 
print(ms_out[0][0])

In [ ]:
class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    # NN foward function
    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            # B: batch, T: Time C: Channel of tensor
            B, T, C = logits.shape
            # reshape the logits tensor from (B, T, C) to (B*T, C), i.e. from (4,8,65) to (32,65) 
            logits = logits.view(B*T, C)
            # reshape the target tensor from (B, T) to (B*T), i.e. from (4,8) to (32) 
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions， self(idx)等同于self.forward(idx)
            logits, loss = self(idx)
            # focus only on the last time step， 得到一个形状为 (B, C) 的二维张量，它只包含原始logits张量中每个批次的最后一个时间步的logits。
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1) 为每个输入批次随机选择一个词，其选择的概率与probs中对应词的概率成正比。
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx
        
# create a model instance, vocab_size is # of token
m = BigramLanguageModel(vocab_size)

# feed the train batches *xb, yb), output is logits and loss
logits, loss = m(xb, yb)
print(f"logits shape : {logits.shape}")
print(f"loss = {loss}")

#print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

TK_NUM = 100
IN_ROW = 2
IN_COL = 2 

input = torch.zeros((IN_ROW, IN_COL), dtype=torch.long)
# print(input[0][0])

for r in range(IN_ROW):
    print(f"The ROW {r} output")
    gen_list = m.generate(input, max_new_tokens=TK_NUM)[r].tolist()
    print(f"gen_list = {gen_list}")
    print(f"generated {len(gen_list)} output")
    print(decode(gen_list))
    print("\n")

## Now let us add an optimizer and train the model, using GPU if avaliable

## These is no attention mechinism yet

In [ ]:
# create a PyTorch optimizer
# optimizer = torch.optim.SGD(m.parameters(), lr=1e-3)
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [ ]:
train_batch_size = 32
train_block_size = 16
for e in range(2000): # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train', train_batch_size, train_block_size)

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    if e % 100 == 0:
        print(f"epoch[{e}]: loss={loss.item()}")

print(loss.item())


In [ ]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))

## The mathematical trick in self-attention

In [ ]:
a_ = torch.tril(torch.ones(3, 3, 3))  # generate a 3x3 triangle matrix
print( a_ )
print( torch.sum(a_, 0, keepdim=True) ) 
print( torch.sum(a_, 1, keepdim=True) )
print( torch.sum(a_, 2, keepdim=True) ) 


In [ ]:
# toy example illustrating how matrix multiplication can be used for a "weighted aggregation"
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))  # generate a 3x3 triangle matrix
print('initial a=')
print(a)
print('divide by:')
print( torch.sum(a, 1, keepdim=True) )  #对matrix a, 沿第二维度(1：列方向）进行求和，keepdim=True: 保留（保持）被求和操作的维度
print('--')
a = a / torch.sum(a, 1, keepdim=True)  # we just do the averaging (normalization) to each row
b = torch.randint(0,10,(3,2)).float()  # generate a 3x2 matrix. value between 0 and 10
c = a @ b # matrix multiply
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)  # c is effectively the average of b on collum level, same result as below

In [ ]:
# consider the following toy example:
torch.manual_seed(SEED)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape
#x

In [ ]:
# xbow: bag of words
# We want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C))

In [ ]:
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)

In [ ]:
xbow

In [ ]:
# version 2: using matrix multiply for a weighted aggregation
wei = torch.tril(torch.ones(T, T))
# print(wei)
wei = wei / wei.sum(1, keepdim=True)
# print(wei)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)

print( torch.allclose(xbow, xbow2, atol=1e-05, rtol=1e-05) ) # atol比较时允许的最大绝对误差 rtol: 相对于较大值的允许的最大相对误差

# print(xbow - xbow2)

In [ ]:
# version 3: use Softmax
tril = torch.tril(torch.ones(T, T))  # a 8x8 triangular matrix
wei = torch.zeros((T,T))             # a 8x8 triangular matrix, all 0
wei = wei.masked_fill(tril == 0, float('-inf')) # mask tril, fo rall 0 to be -inf
wei = F.softmax(wei, dim=-1)  # softmax the wei
xbow3 = wei @ x  # weight the x

# print(xbow2 == xbow3)
torch.allclose(xbow2 , xbow3)


In [ ]:
# version 3: use Softmax -- step by step
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
print(tril)
print(wei)

In [ ]:
wei = wei.masked_fill(tril == 0, float('-inf'))
print(wei)

In [ ]:
wei = F.softmax(wei, dim=-1) 
print(wei)

In [ ]:
xbow3 = wei @ x 
print(xbow3)

### self-attention, with single head

#### 准备input token

In [ ]:
torch.manual_seed(SEED)
B,T,C = 4,8,32 # 4 batched, each batch has 4 token, each token is 32 dim
x = torch.randn(B,T,C)  # 4x8x32

print(x.shape) # original token
print(x) # original token

#### 使用FC layer (nn.Liner) 准备K, Q, V 层

In [ ]:
head_size = 16
key   = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)

print(f"key shape {key.weight.shape}, query shape {query.weight.shape}, value shape {value.weight.shape}")

#### pass token through K, Q, calculate W = QK^T

In [ ]:
k = key(x)   # (B, T, 16)  pass x (B,T,C) to key^T (Cx16), k will be (B, T, 16)
q = query(x) # (B, T, 16)

以 k = key(x) 为例，当 x 通过 key 层时，实际上是进行了以下矩阵乘法(线性变换）：
$$
key(x) = \text{key.weight} \times \text{reshape}(\text{x}) + \text{key.bias}
$$
其中 $reshape(x)$ 将 x 重塑(类似扁平化排列)为一个二维矩阵，其形状为 (B\*T,C)，然后与权重矩阵 key.weight（形状为 (head_size, C)）相乘，最后再加上偏置项（如果有的话）。由于 bias=False，所以实际上没有偏置项。
在GPU上进行计算时，输入张量 x 形状为 (B, T, C) 会被视作一个 (B*T, C) 的二维矩阵，而 key 层的权重则被视作一个 (C, head_size) 的矩阵。然后，这两个矩阵进行矩阵乘法运算：

这个矩阵乘法的结果是 (B*T, head_size) 的矩阵。然后，这个结果可以被重新塑形（reshape）回 (B, T, head_size) 的张量形式，以匹配原始批次和时间步的维度。

这种操作在深度学习框架中非常常见，它们利用了矩阵乘法的并行性，可以非常高效地在GPU上执行。
这种扁平化和重塑的过程是自动进行的，用户不需要手动干预，深度学习框架会处理这些细节，以确保计算的高效性和正确性。

In [ ]:
wei =  q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T) -2 和 -1 是索引，它们分别表示倒数第二个和最后一个维度。.transpose(-2, -1) 将这两个维度互换位置，

在执行 wei = q @ k.transpose(-2, -1) 这个操作时，实际上是对每个批次 B 中的矩阵进行并行的矩阵乘法。

具体来说：

q 的形状是 (B, T, head_size)，其中 B 是批次大小，T 是时间步长，head_size 是头的输出维度。
k.transpose(-2, -1) 交换了最后一个维度(-1)和倒数第二个维度(-2)，实际上等同于B个矩阵分别转置。

矩阵乘法 q @ k.transpose(-2, -1) 是这样进行的：

- q实际上是B个形状为(T, head_size)的矩阵，
- k.transpose(-2, -1) 实际上是B个形状为(head_size，T)的矩阵，

矩阵乘法发生在 q 的最后一个维度 head_size 和 k.transpose(-2, -1) 的第二个维度 head_size 之间。
结果是一个 (B, T, T) 形状的矩阵。
这个操作可以看作是 B 个 (T, head_size) 和 (T, head_size) 的矩阵乘法并行执行。对于每个批次 b，都有一个独立的 (T, T) 矩阵计算，B个矩阵的乘法计算是同时进行的，利用了现代深度学习框架的并行计算能力。

In [ ]:
print(f"wei shape {wei.shape}")
print(f"{wei}")

#### calculate W=softmax(W)

In [ ]:
tril = torch.tril(torch.ones(T, T))  # mask matrix, make sure future token value is not be impacted
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
print(f"Weight shape {wei.shape}")
print(f"{wei}")

#### pass token through V, calculate WV

In [ ]:
v = value(x)     # pass x (B,T,C) to value^T (Cx16), v will be (B, T, 16)
out = wei @ v    # wei

print(f"wei.shape {wei.shape}, v.shape {v.shape}, out.shape {out.shape}")
print(f"{out}")

Notes:
- Attention is a **communication mechanism**. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.
- There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens.
- Each example across batch dimension is of course processed completely independently and never "talk" to each other
- In an "encoder" attention block just delete the single line that does masking with `tril`, allowing all tokens to communicate. This block here is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modeling.
- "self-attention" just means that the keys and values are produced from the same source as queries. In "cross-attention", the queries still get produced from x, but the keys and values come from some other, external source (e.g. an encoder module)
- "Scaled" attention additional divides `wei` by 1/sqrt(head_size). This makes it so when input Q,K are unit variance, wei will be unit variance too and Softmax will stay diffuse and not saturate too much. Illustration below

#### Why using  "Scale" operation 

In [ ]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2, -1) * head_size**-0.5

In [ ]:
k.var()

In [ ]:
q.var()

In [ ]:
wei.var()

In [ ]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1)

In [ ]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5])*8, dim=-1) # gets too peaky, converges to one-hot

In [ ]:
class LayerNorm1d: # (used to be BatchNorm1d)

  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)

  def __call__(self, x):
    # calculate the forward pass
    xmean = x.mean(1, keepdim=True) # batch mean
    xvar = x.var(1, keepdim=True) # batch variance
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    return self.out

  def parameters(self):
    return [self.gamma, self.beta]

torch.manual_seed(SEED)
module = LayerNorm1d(100)
x = torch.randn(32, 100) # batch size 32 of 100-dimensional vectors
x = module(x)
x.shape

In [ ]:
x[:,0].mean(), x[:,0].std() # mean,std of one feature across all batch inputs

In [ ]:
x[0,:].mean(), x[0,:].std() # mean,std of a single input from the batch, of its features

In [ ]:
# French to English translation example:

# <--------- ENCODE ------------------><--------------- DECODE ----------------->
# les réseaux de neurones sont géniaux! <START> neural networks are awesome!<END>



### Full finished code, for reference

You may want to refer directly to the git repo instead though.

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
# max_iters = 1000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

torch.manual_seed(SEED)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss(model):
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx


# Train, test and load pre-trained model

In [ ]:
def load_pretrained_model(pre_train_model: str):
    model = BigramLanguageModel()  # 初始化模型
    model.load_state_dict(torch.load(pre_train_model))
    model.eval()  # 设置为评估模式
    return model
def train_model(name: str, iter:int, pre_train: str= None):
    
    if pre_train == None:
        model = BigramLanguageModel()
    else:
        model = load_pretrained_model(pre_train)
        
    m = model.to(device)
    # print the number of parameters in the model
    print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')
    
    # create a PyTorch optimizer
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
    
    for iter in range(iter):
    
        # every once in a while evaluate the loss on train and val sets
        if iter % eval_interval == 0 or iter == iter - 1:
            losses = estimate_loss(m)
            print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
    
        # sample a batch of data
        xb, yb = get_batch('train')
    
        # evaluate the loss
        logits, loss = model(xb, yb)
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()
    
    model_name = f"{name}.pth"
    # 保存模型的状态字典
    torch.save(m.state_dict(), model_name)
    print(f"model is saved as {model_name}")
    return m
    
def test_model(m, max_token):  
    # generate from the model
    context = torch.zeros((1, 1), dtype=torch.long, device=device)
    print(decode(m.generate(context, max_new_tokens=max_token)[0].tolist()))


In [ ]:
my_model = train_model("first_1k", 1000, None)

In [ ]:
test_model(my_model, 300)

In [ ]:
my_model = train_model("second_1k", 1000, 'first_1k.pth')

In [ ]:
test_model(my_model, 300)

In [ ]:
my_model = train_model("third_1k", 1000, 'second_1k.pth')

In [ ]:
my_model = train_model("fourth_1k", 1000, 'third_1k.pth')

In [ ]:
my_model = train_model("fifth_5k", 5000, 'fourth_1k.pth') 